<a href="https://colab.research.google.com/github/T0mSIlver/OCT_GANs/blob/main/cycleGAN/cycleGAN_oct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

oct and dfe are pretty different, patches may pose a problem
results are not very good

# Imports

In [1]:
from PIL import Image
import os
import random
import numpy as np
import shutil
import zipfile
from google.colab import drive

import torch.nn as nn
import copy

import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
import sys
import torch.optim as optim
from tqdm import tqdm
from torchvision.utils import save_image

# Data preparation

## connect drive and kaggle

In [2]:
drive.mount('/content/drive')
drive_path = "/content/drive/My Drive/Colab/progan/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
kaggle_key = '/content/drive/MyDrive/Colab/progan/kaggle.json'
shutil.copyfile(kaggle_key, '/content/kaggle.json')

'/content/kaggle.json'

In [4]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
! kaggle datasets download "andrewmvd/ocular-disease-recognition-odir5k"

ocular-disease-recognition-odir5k.zip: Skipping, found more recently modified local copy (use --force to force download)


## create directories

In [6]:
! mkdir /content/data
! mkdir /content/data/val
! mkdir /content/data/train

mkdir: cannot create directory ‘/content/data’: File exists
mkdir: cannot create directory ‘/content/data/val’: File exists
mkdir: cannot create directory ‘/content/data/train’: File exists


In [7]:
! mkdir /content/data/train/dfe
! mkdir /content/data/train/oct
! mkdir /content/data/val/dfe
! mkdir /content/data/val/oct

mkdir: cannot create directory ‘/content/data/train/dfe’: File exists
mkdir: cannot create directory ‘/content/data/train/oct’: File exists
mkdir: cannot create directory ‘/content/data/val/dfe’: File exists
mkdir: cannot create directory ‘/content/data/val/oct’: File exists


## copy files

### oct

In [ ]:
! rm -r "/content/data/train/oct/"
! rm -r "/content/data/val/oct/"

In [8]:
zip_ref = zipfile.ZipFile(drive_path+'kermany2018.zip', 'r')
nb_samples = 2000
nb_samples_val = 20
counter = 0
counter_val=0
for file in zip_ref.infolist():
    if file.filename.startswith('OCT2017 /train/NORMAL') and counter < nb_samples: #ONLY DOWNLOADING DRUSEN train IMAGES
        #print(file.filename.split('/')[-1])
        zip_ref.extract(file)
        counter += 1
    if file.filename.startswith('OCT2017 /val/NORMAL') and counter_val < nb_samples_val:
        zip_ref.extract(file)
        counter += 1
zip_ref.close()

In [9]:
directory = '/content/OCT2017 /val/NORMAL/'
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
      #print('saving', filename, ' ...')
      shutil.copyfile(directory+filename, '/content/data/val/oct/'+filename)
print("complete")

complete


In [10]:
directory = '/content/OCT2017 /train/NORMAL/'
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
      #print('saving', filename, ' ...')
      shutil.copyfile(directory+filename, '/content/data/train/oct/'+filename)
print("complete")

complete


In [11]:
! rm -r "/content/OCT2017 /"

### dfe

In [12]:
zip_ref = zipfile.ZipFile('ocular-disease-recognition-odir5k.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [13]:
import pandas as pd

In [14]:
df = pd.read_excel('/content/ODIR-5K/ODIR-5K/data.xlsx')

In [15]:
for image in df[df.N == 1]['Right-Fundus'].to_list()[:nb_samples]:
  shutil.copyfile('/content/ODIR-5K/ODIR-5K/Training Images/'+image, '/content/data/train/dfe/'+image)

In [16]:
directory = '/content/ODIR-5K/ODIR-5K/Testing Images/'
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
      #print('saving', filename, ' ...')
      shutil.copyfile(directory+filename, '/content/data/val/dfe/'+filename)
print("complete")

complete


In [17]:
! rm -r "/content/ODIR-5K/"

# Config

In [18]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TRAIN_DIR = "data/train"
VAL_DIR = "data/val"
BATCH_SIZE = 1
LEARNING_RATE = 1e-5
LAMBDA_IDENTITY = 0.0
LAMBDA_CYCLE = 10
NUM_WORKERS = 2
NUM_EPOCHS = 10
LOAD_MODEL = False
SAVE_MODEL = True
CHECKPOINT_GEN_H = "genh.pth.tar"
CHECKPOINT_GEN_Z = "genz.pth.tar"
CHECKPOINT_CRITIC_H = "critich.pth.tar"
CHECKPOINT_CRITIC_Z = "criticz.pth.tar"

transforms = A.Compose(
    [
        A.Resize(width=256, height=256),
        A.CenterCrop(width=256, height=256),
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255),
        ToTensorV2(),
    ],
    additional_targets={"image0": "image"},
    is_check_shapes=False,
)


In [19]:
def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


def seed_everything(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Dataset class

In [20]:
class DfeOctDataset(Dataset):
    def __init__(self, root_oct, root_dfe, transform=None):
        self.root_oct = root_oct
        self.root_dfe = root_dfe
        self.transform = transform

        self.oct_images = os.listdir(root_oct)
        self.dfe_images = os.listdir(root_dfe)
        self.length_dataset = max(len(self.oct_images), len(self.dfe_images)) # 1000, 1500
        self.oct_len = len(self.oct_images)
        self.dfe_len = len(self.dfe_images)

    def __len__(self):
        return self.length_dataset

    def __getitem__(self, index):
        oct_img = self.oct_images[index % self.oct_len]
        dfe_img = self.dfe_images[index % self.dfe_len]

        oct_path = os.path.join(self.root_oct, oct_img)
        dfe_path = os.path.join(self.root_dfe, dfe_img)

        oct_img = np.array(Image.open(oct_path).convert("RGB"))
        dfe_img = np.array(Image.open(dfe_path).convert("RGB"))

        if self.transform:
            augmentations = self.transform(image=oct_img, image0=dfe_img)
            oct_img = augmentations["image"]
            dfe_img = augmentations["image0"]

        return oct_img, dfe_img

# Discriminator

In [21]:
"""
Discriminator model for CycleGAN

Programmed by Aladdin Persson <aladdin.persson at hotmail dot com>
* 2020-11-05: Initial coding
* 2022-12-21: Small revision of code, checked that it works with latest PyTorch version
"""

class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                4,
                stride,
                1,
                bias=True,
                padding_mode="reflect",
            ),
            nn.InstanceNorm2d(out_channels),
            nn.LeakyReLU(0.2, inplace=True),
        )

    def forward(self, x):
        return self.conv(x)


class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 128, 256, 512]):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(
                in_channels,
                features[0],
                kernel_size=4,
                stride=2,
                padding=1,
                padding_mode="reflect",
            ),
            nn.LeakyReLU(0.2, inplace=True),
        )

        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            layers.append(
                Block(in_channels, feature, stride=1 if feature == features[-1] else 2)
            )
            in_channels = feature
        layers.append(
            nn.Conv2d(
                in_channels,
                1,
                kernel_size=4,
                stride=1,
                padding=1,
                padding_mode="reflect",
            )
        )
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = self.initial(x)
        return torch.sigmoid(self.model(x))


def test():
    x = torch.randn((5, 3, 256, 256))
    model = Discriminator(in_channels=3)
    preds = model(x)
    print(preds.shape)


if __name__ == "__main__":
    test()

torch.Size([5, 1, 30, 30])


# Generator

In [22]:
"""
Generator model for CycleGAN

Programmed by Aladdin Persson <aladdin.persson at hotmail dot com>
* 2020-11-05: Initial coding
* 2022-12-21: Small revision of code, checked that it works with latest PyTorch version
"""

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, use_act=True, **kwargs):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, padding_mode="reflect", **kwargs)
            if down
            else nn.ConvTranspose2d(in_channels, out_channels, **kwargs),
            nn.InstanceNorm2d(out_channels),
            nn.ReLU(inplace=True) if use_act else nn.Identity(),
        )

    def forward(self, x):
        return self.conv(x)


class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            ConvBlock(channels, channels, kernel_size=3, padding=1),
            ConvBlock(channels, channels, use_act=False, kernel_size=3, padding=1),
        )

    def forward(self, x):
        return x + self.block(x)


class Generator(nn.Module):
    def __init__(self, img_channels, num_features=64, num_residuals=9):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(
                img_channels,
                num_features,
                kernel_size=7,
                stride=1,
                padding=3,
                padding_mode="reflect",
            ),
            nn.InstanceNorm2d(num_features),
            nn.ReLU(inplace=True),
        )
        self.down_blocks = nn.ModuleList(
            [
                ConvBlock(
                    num_features, num_features * 2, kernel_size=3, stride=2, padding=1
                ),
                ConvBlock(
                    num_features * 2,
                    num_features * 4,
                    kernel_size=3,
                    stride=2,
                    padding=1,
                ),
            ]
        )
        self.res_blocks = nn.Sequential(
            *[ResidualBlock(num_features * 4) for _ in range(num_residuals)]
        )
        self.up_blocks = nn.ModuleList(
            [
                ConvBlock(
                    num_features * 4,
                    num_features * 2,
                    down=False,
                    kernel_size=3,
                    stride=2,
                    padding=1,
                    output_padding=1,
                ),
                ConvBlock(
                    num_features * 2,
                    num_features * 1,
                    down=False,
                    kernel_size=3,
                    stride=2,
                    padding=1,
                    output_padding=1,
                ),
            ]
        )

        self.last = nn.Conv2d(
            num_features * 1,
            img_channels,
            kernel_size=7,
            stride=1,
            padding=3,
            padding_mode="reflect",
        )

    def forward(self, x):
        x = self.initial(x)
        for layer in self.down_blocks:
            x = layer(x)
        x = self.res_blocks(x)
        for layer in self.up_blocks:
            x = layer(x)
        return torch.tanh(self.last(x))


def test():
    img_channels = 3
    img_size = 256
    x = torch.randn((2, img_channels, img_size, img_size))
    gen = Generator(img_channels, 9)
    print(gen(x).shape)


if __name__ == "__main__":
    test()

torch.Size([2, 3, 256, 256])


# Training loop

In [26]:
! mkdir /content/saved_images

In [23]:
"""
Training for CycleGAN

Programmed by Aladdin Persson <aladdin.persson at hotmail dot com>
* 2020-11-05: Initial coding
* 2022-12-21: Small revision of code, checked that it works with latest PyTorch version
"""

def train_fn(
    disc_H, disc_Z, gen_Z, gen_H, loader, opt_disc, opt_gen, l1, mse, d_scaler, g_scaler, epoch
):
    H_reals = 0
    H_fakes = 0
    loop = tqdm(loader, leave=True)

    for idx, (oct, dfe) in enumerate(loop):
        oct = oct.to(DEVICE)
        dfe = dfe.to(DEVICE)

        # Train Discriminators H and Z
        with torch.cuda.amp.autocast():
            fake_dfe = gen_H(oct)
            D_H_real = disc_H(dfe)
            D_H_fake = disc_H(fake_dfe.detach())
            H_reals += D_H_real.mean().item()
            H_fakes += D_H_fake.mean().item()
            D_H_real_loss = mse(D_H_real, torch.ones_like(D_H_real))
            D_H_fake_loss = mse(D_H_fake, torch.zeros_like(D_H_fake))
            D_H_loss = D_H_real_loss + D_H_fake_loss

            fake_oct = gen_Z(dfe)
            D_Z_real = disc_Z(oct)
            D_Z_fake = disc_Z(fake_oct.detach())
            D_Z_real_loss = mse(D_Z_real, torch.ones_like(D_Z_real))
            D_Z_fake_loss = mse(D_Z_fake, torch.zeros_like(D_Z_fake))
            D_Z_loss = D_Z_real_loss + D_Z_fake_loss

            # put it togethor
            D_loss = (D_H_loss + D_Z_loss) / 2

        opt_disc.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(opt_disc)
        d_scaler.update()

        # Train Generators H and Z
        with torch.cuda.amp.autocast():
            # adversarial loss for both generators
            D_H_fake = disc_H(fake_dfe)
            D_Z_fake = disc_Z(fake_oct)
            loss_G_H = mse(D_H_fake, torch.ones_like(D_H_fake))
            loss_G_Z = mse(D_Z_fake, torch.ones_like(D_Z_fake))

            # cycle loss
            cycle_oct = gen_Z(fake_dfe)
            cycle_dfe = gen_H(fake_oct)
            cycle_oct_loss = l1(oct, cycle_oct)
            cycle_dfe_loss = l1(dfe, cycle_dfe)

            # identity loss (remove these for efficiency if you set lambda_identity=0)
            identity_oct = gen_Z(oct)
            identity_dfe = gen_H(dfe)
            identity_oct_loss = l1(oct, identity_oct)
            identity_dfe_loss = l1(dfe, identity_dfe)

            # add all togethor
            G_loss = (
                loss_G_Z
                + loss_G_H
                + cycle_oct_loss * LAMBDA_CYCLE
                + cycle_dfe_loss * LAMBDA_CYCLE
                + identity_dfe_loss * LAMBDA_IDENTITY
                + identity_oct_loss * LAMBDA_IDENTITY
            )

        opt_gen.zero_grad()
        g_scaler.scale(G_loss).backward()
        g_scaler.step(opt_gen)
        g_scaler.update()

        if idx % 200 == 0:
            save_image(fake_dfe * 0.5 + 0.5, f"saved_images/dfe_{epoch}_{idx}.png")
            save_image(fake_oct * 0.5 + 0.5, f"saved_images/oct_{epoch}_{idx}.png")

        loop.set_postfix(H_real=H_reals / (idx + 1), H_fake=H_fakes / (idx + 1))



# Main

In [24]:
def main():
    disc_H = Discriminator(in_channels=3).to(DEVICE)
    disc_Z = Discriminator(in_channels=3).to(DEVICE)
    gen_Z = Generator(img_channels=3, num_residuals=9).to(DEVICE)
    gen_H = Generator(img_channels=3, num_residuals=9).to(DEVICE)
    opt_disc = optim.Adam(
        list(disc_H.parameters()) + list(disc_Z.parameters()),
        lr=LEARNING_RATE,
        betas=(0.5, 0.999),
    )

    opt_gen = optim.Adam(
        list(gen_Z.parameters()) + list(gen_H.parameters()),
        lr=LEARNING_RATE,
        betas=(0.5, 0.999),
    )

    L1 = nn.L1Loss()
    mse = nn.MSELoss()

    if LOAD_MODEL:
        load_checkpoint(
            CHECKPOINT_GEN_H,
            gen_H,
            opt_gen,
            LEARNING_RATE,
        )
        load_checkpoint(
            CHECKPOINT_GEN_Z,
            gen_Z,
            opt_gen,
            LEARNING_RATE,
        )
        load_checkpoint(
            CHECKPOINT_CRITIC_H,
            disc_H,
            opt_disc,
            LEARNING_RATE,
        )
        load_checkpoint(
            CHECKPOINT_CRITIC_Z,
            disc_Z,
            opt_disc,
            LEARNING_RATE,
        )

    dataset = DfeOctDataset(
        root_dfe=TRAIN_DIR + "/dfe",
        root_oct=TRAIN_DIR + "/oct",
        transform=transforms,
    )
    val_dataset = DfeOctDataset(
        root_dfe= VAL_DIR + "/dfe",
        root_oct= VAL_DIR + "/oct",
        transform=transforms,
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=1,
        shuffle=False,
        pin_memory=True,
    )
    loader = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True,
    )
    g_scaler = torch.cuda.amp.GradScaler()
    d_scaler = torch.cuda.amp.GradScaler()

    for epoch in range(NUM_EPOCHS):
        train_fn(
            disc_H,
            disc_Z,
            gen_Z,
            gen_H,
            loader,
            opt_disc,
            opt_gen,
            L1,
            mse,
            d_scaler,
            g_scaler,
            epoch,
        )

        if SAVE_MODEL:
            save_checkpoint(gen_H, opt_gen, filename=CHECKPOINT_GEN_H)
            save_checkpoint(gen_Z, opt_gen, filename=CHECKPOINT_GEN_Z)
            save_checkpoint(disc_H, opt_disc, filename=CHECKPOINT_CRITIC_H)
            save_checkpoint(disc_Z, opt_disc, filename=CHECKPOINT_CRITIC_Z)

In [27]:
main()

100%|██████████| 2000/2000 [08:27<00:00,  3.94it/s, H_fake=0.28, H_real=0.713]


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 2000/2000 [08:38<00:00,  3.86it/s, H_fake=0.197, H_real=0.804]


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 2000/2000 [08:29<00:00,  3.93it/s, H_fake=0.171, H_real=0.833]


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 2000/2000 [08:28<00:00,  3.93it/s, H_fake=0.169, H_real=0.838]


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


 11%|█         | 221/2000 [00:56<07:37,  3.89it/s, H_fake=0.174, H_real=0.832]


KeyboardInterrupt: ignored

images probably too different

# Save images to drive

In [30]:
directory = '/content/saved_images/'
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
      #print('saving', filename, ' ...')
      shutil.copyfile(directory+filename, "/content/drive/My Drive/Colab/cycleGAN/"+filename)
print("complete")

complete
